In [3]:

import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

from demosearch import FileCache
from pathlib import Path

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

In [4]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()

pkg_root = Path(pkg.path).parent
cache = FileCache(pkg_root.joinpath('data/cache'))

pkg

# San Diego Business Clusters
`sandiegodata.org-business_clusters-1.3.3` Last Update: 2021-03-26T16:01:10

_Business in San Diego linked to entertainment clusters and population density._


This dataset processes the City of San Diego Master Business file to add
geocoded addresses and links to business clusters. San Diego publishes two
lists of businesses, which are based on payment of the San Diego City business
tax: the Master Business File, and a [SANGIS file that includes geographic
information.
](https://data.sandiegodata.org/dataset/sangis-org-business-sites/) Unfortunatel
y, these files are quite different and cannot be linked. The SANGIS file is
oriented toward the tax assessors parcel that the business occupies, and the
Master Business List has account numbers and addresses, but there is no common
key between the files.

The files in this package add address geocodes to the Master Business List, and
links the businesses to clusters of businesses. The Clusters are created by
collecting nearby businesses from Open Street Map data. The cluster types are:

* NA: No cluster, 31787 businesses
* shop: OSM tags 'shop', 'clothes', 'supermarket', 'bank', 'laundry', 'parking',  14615 businesses
* ent: Entertainment, OSM tags 'cafe', 'restaurant', 'bar', 14320 businesses
* casual: Fast food and convenience stores, OSM tags  'fast_food', 'convenience', 10991 businesses

The ``sd_business_clusters`` file has the clusters and their WKT geographies.
The ``sd_custered_businesses`` links San Diego businesses to clusters, and a
single business may be in more than one cluster because the clusters of
different types overlap. The `sd_ucustered_businesses` has the businesses deduplicated, with a business that is in multiple clusters taking only one cluster, in this order, from highest to lowest preference: 'ent', 'shop', 'casual','NA'

## NAICS Codes

It appears that the  NAICS codes used in the Master Business List are vintage 2007. The code '72221' appears frequently, which is valid in 2007 NAICS, but not in 2012 or 2016 NAICS. 

## Geocoding

The geocoding was performed with a local installation of Pelias. There are some notable errors in the geocoding. For instance, Ba Ho Liquor and Deli, with address of '4031 AVATI DR SUITE I SAN DIEGO 92117-4403, CA', was geocoded to 4144 Avati, moving the location from a neighborhood mini-mall to a residence. It is unknown how many such error there are, so use the geocodes with caution.


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [sb_mbl](notebooks/Extract.ipynb#sb_mbl)**. Businesses registered in San DIego, from the Master BusinessList
* ** [sd_business_clusters](notebooks/Extract.ipynb#sd_business_clusters)**. Geographic boundaries of business clusters
* ** [sd_businesses](notebooks/Extract.ipynb#sd_businesses)**. San Diego Businesses, geocoded
* ** [sd_clustered_businesses](notebooks/Extract.ipynb#sd_custered_businesses)**. San Diego Businesses, geocodes and linked to clusters. Businesses in multiple clusters are duplicated
* ** [naics](notebooks/Extract.ipynb#naics)**. Naics codes for San Diego businesses
* ** [blockgroups](notebooks/Extract.ipynb#blocks)**. Geo data for census block groups and total population
* ** [bid](notebooks/Extract.ipynb#bid)**. San Diego Business Improvement Districts

## References
<ul><li> <strong>business_clusters</string>, <em>index:civicknowledge.com-osm-demosearch-2.1.1#business_clusters</em>. US business clusters</li><li> <strong><a href="https://www.sandiego.gov/sites/default/files/tr_active1.csv">sd_businesses_ak</a></strong>. San Diego Businesses A-K</li><li> <strong><a href="https://www.sandiego.gov/sites/default/files/tr_active2.csv">sd_businesses_lz</a></strong>. San Diego Businesses L-Z</li><li> <strong>business_sites</string>, <em>metapack+http://library.metatab.org/sangis.org-business_sites.csv#business_sites</em>. San DIego Business locations, from SANGI

In [3]:
pkg.reference('naics_index_2007_26').dataframe()

,Seq. No.,2017 NAICS US Code,2017 NAICS US Title,,,
0,,,,,,
1,1.0,11.0,"Agriculture, Forestry, Fishing and Hunting",,,
2,2.0,111.0,Crop Production,,,
3,3.0,1111.0,Oilseed and Grain Farming,,,
4,4.0,11111.0,Soybean Farming,,,
...,...,...,...,...,...,...
2192,2192.0,9281.0,National Security and International Affairs,,,
2193,2193.0,92811.0,National Security,,,
2194,2194.0,928110.0,National Security,,,
2195,2195.0,92812.0,International Affairs,,,


In [4]:
t = pkg.reference('naics_index_2007_26').dataframe()
t.columns = ['seq','naics_code','naics_desc','a','b','c']
naics = t

def int_maybe(v):
    try:
        return int(v)
    except:
        return v

naics_map = { int_maybe(r.naics_code):r.naics_desc for idx, r in naics.iterrows()}

In [5]:
sd_county = pkg.reference('sd_county_boundary').geoframe()
bs = pkg.reference('business_sites').geoframe()
bc = pkg.reference('business_clusters').geoframe()

In [6]:
sdb_ak = pkg.reference('sd_businesses_ak').dataframe()
sdb_lz = pkg.reference('sd_businesses_lz').dataframe()

sdb = pd.concat([sdb_ak, sdb_lz])
sb_mbl = sdb # The name in the metadata
sdb.head()

,BUSINESS ACCT#,DBA NAME,OWNERSHIP TYPE,ADDRESS,CITY,ZIP,STATE,BUSINESS PHONE,OWNER NAME,CREATION DT,START DT,EXP DT,NAICS,ACTIVITY DESC
0,2020002344,#0205,LLC,4704 IMPERIAL AVE,SAN DIEGO,92113-5001,CA,(619) 436-5006,CF UNITED CVX HOLDINGS LLC,02/07/2020,11/14/2019,11/30/2021,44711,GASOLINE STATIONS WITH CONVENIENCE STORES
1,2019021709,#1 FIFTH AVE,LLC,3845 05TH AVE,SAN DIEGO,92103-3140,CA,(619) 299-1911,ICONIC BAR INVESTMENTS LLC,10/15/2019,10/11/2019,10/31/2021,72241,DRINKING PLACES (ALCOHOLIC BEVERAGES)
2,2019010605,#OOHMAMAMACARONS,SOLE,515 S SAN JACINTO DR,SAN DIEGO,92114-5331,CA,NaN,YESENIA RAMIREZ,05/17/2019,05/16/2019,05/31/2021,72234,COTTAGE FOOD OPERATOR
3,2015046247,& SALLY J FERNANDEZ,SOLE,17471 PLAZA CERADO SUITE 98,SAN DIEGO,92128-2285,CA,NaN,JAIME V FERNANDEZ,10/29/2015,01/01/2014,12/31/2021,541219,OTHER ACCOUNTING SERVICES
4,2018011836,(W)RIGHT ON COMMUNICATIONS INC,SCORP,1200 PROSPECT ST SUITE G100,LA JOLLA,92037-3608,CA,(858) 866-7900,(W)RIGHT ON COMMUNICATIONS INC,06/22/2018,06/01/2018,05/31/2021,54182,PUBLIC RELATIONS AGENCIES


In [7]:
nl = pd.DataFrame( {
    'account' :sb_mbl['BUSINESS ACCT#'],
    'naics'   :sb_mbl['NAICS'],
    'naics_2' :sb_mbl['NAICS'].astype(str).str.slice(0,2),
    'naics_3' :sb_mbl['NAICS'].astype(str).str.slice(0,3),
    'naics_4' :sb_mbl['NAICS'].astype(str).str.slice(0,4),
    'naics_5' :sb_mbl['NAICS'].astype(str).str.slice(0,5),
    'naics_6' :sb_mbl['NAICS'].astype(str).str.slice(0,6),
})

for c in nl.columns[1:]:
    dc = c+"_desc"
    nl[dc] = nl[c].apply(lambda v: naics_map.get(int_maybe(v)) )

naics = nl

In [8]:
naics_map.get(72241)

'Drinking Places (Alcoholic Beverages)'

In [9]:
t  = gpd.sjoin(bc, sd_county)
sd_business_clusters = t[['cluster_n','geometry']]

In [6]:
from demosearch.geocode import CachingGeocoder

gc = CachingGeocoder(cache)

def geocode(cache, text):
    
    text_enc = urllib.parse.quote_plus(text)
    text_h = hashlib.sha224(text_enc.encode('utf8')).hexdigest()
    
    key = f"geocode/{text_h[0:2]}/{text_h[2:4]}/{text_h}"
    
    if cache.exists(key):
        return cache.get(key)
    else:
        r = requests.get('http://barker:4000/v1/search?text='+text_enc)
        r.raise_for_status()
        j = r.json()
        
        cache.put(key, j)
        return j

gc.geocode('1370 Wilbur Ave, San Diego, CA')
    

Status code Unknown from http://barker:8080: ERROR - HTTPConnectionPool(host='barker', port=8080): Read timed out. (read timeout=5.0)


!!! 1370 Wilbur Ave, San Diego, CA <[ERROR - HTTPConnectionPool(host='barker', port=8080): Read timed out. (read timeout=5.0)] Osm - Geocode [empty]>


GeocodeFailed: Failed to geocode '1370 Wilbur Ave, San Diego, CA'

In [10]:
#http://barker:4000/v1/search?text=1370%20Wilbur,%20San%20Diego,%20CA
import requests
import urllib.parse
import hashlib 
from demosearch.util import run_mp

    
def geocode_chunk(cache, frame_key):
    from time import sleep
    from random import uniform
    
        
    df = cache.get(frame_key)
    results = []
    for idx, row in df.iterrows():
        addr = f"{row.ADDRESS} {row.CITY} {row.ZIP}, {row.STATE}"
    
        sleep_time = 1
        for i in range(3):
    
            try:
                j = geocode(cache, addr)
                r = j['features'][0]['geometry']
            except IndexError as e:
                
                break
            except Exception as e:
                if e.response is not None and e.response.status_code == 400:
                    print("Timeout: Iter",i,' sleep', sleep_time, e)
                    sleep(sleep_time+uniform(0,3))
                    sleep_time *= 2
                       
                else:
                    r = e
                    break
            
        results.append( (row['BUSINESS ACCT#'], addr, r))

    return results

chunk_size = int(len(sdb)/100)


frames = [sdb[i:i+chunk_size] for i in range(0,sdb.shape[0],chunk_size)]
tasks = []
for i, f in enumerate(frames):
    key =  f'chunks/{i}'
    
    cache.put_df(key, f)
    tasks.append( (cache, key))

print("Start MP run")
r = run_mp(geocode_chunk, tasks, n_cpu=4)

In [13]:
rc = [ e for e in list(chain(*r))]

In [22]:
rc[0][2]

requests.exceptions.ConnectionError(urllib3.exceptions.MaxRetryError('None: Max retries exceeded with url: /v1/search?text=4704+IMPERIAL+AVE+SAN+DIEGO+92113-5001%2C+CA (Caused by None)'))

In [24]:
sum( 1 if isinstance(e[2], Exception) else 0 for e in rc )/len(rc)

1.0

In [10]:
from itertools import chain
import geopandas as gpd
rows = [ (e[0],e[1],e[2]['coordinates'][1], e[2]['coordinates'][0]) for e in list(chain(*r))]

gcodes = pd.DataFrame(rows, columns='account gc_address lat lon'.split())
gcodes['geometry'] = gpd.points_from_xy(gcodes.lon, gcodes.lat)
gcodes = gpd.GeoDataFrame(gcodes, crs=4326)

t = gcodes.merge(sdb, left_on='account', right_on='BUSINESS ACCT#')
t  = gpd.sjoin(t, sd_county)

Start MP run


  0%|          | 0/101 [00:00<?, ?it/s]

TypeError: 'ConnectionError' object is not subscriptable

In [ ]:
cols = ['account', 'gc_address', 'lat', 'lon',  'DBA NAME', 'OWNERSHIP TYPE',
         'CREATION DT', 'START DT', 'EXP DT', 'OWNER NAME','NAICS', 'ACTIVITY DESC', 'geometry']
sd_businesses = t[cols]

In [ ]:
blocks = pkg.reference('census_bg').geoframe()
pop =  pkg.reference('total_population').dataframe()
t = blocks.merge(pop.reset_index())
cols = ['geoid',  'aland','b01003_001', 'intptlat', 'intptlon', 'geometry']
blocks = t[cols].rename(columns={'b01003_001':'pop', "aland":'area'}).to_crs(4326)


In [ ]:
# Merge blockgroup population into sd_businesses
from geoid.censusnames import stusab
import rowgenerators as rg


t = gpd.sjoin(sd_businesses, blocks)
sd_businesses = t[list(sd_businesses.columns) + ['geoid','pop', 'area']]

sd_businesses.head()

In [ ]:
t = gpd.sjoin(sd_businesses, sd_business_clusters, how='left')
cols = ['account', 'gc_address', 'lat', 'lon', 'geoid', 'pop', 'area', 'DBA NAME', 'OWNERSHIP TYPE',
         'CREATION DT', 'START DT', 'EXP DT', 'OWNER NAME','NAICS', 'ACTIVITY DESC', 
        'cluster_n','geometry']
t['cluster_n'] = t.cluster_n.fillna(0)
sd_custered_businesses = t[cols]


In [ ]:
bid = pkg.reference('bid').geoframe().to_crs(4326)
bid = bid[['NAME2','SYMBOL', 'geometry']].rename(columns={'NAME2':'bid_name','SYMBOL':'bid_code'})
bid.plot(column='bid_name')


In [ ]:
t = gpd.sjoin(sd_custered_businesses, bid, how='left').drop(columns=['index_right'])
sd_custered_businesses = t[ [c for c in t.columns if c != 'geometry'] + ['geometry']]

In [ ]:
sd_custered_businesses.head().T

In [ ]:
!pwd
